In [1100]:
# -*- coding: utf8 -*-
from os.path import expanduser
home = expanduser("~")
repertoire=home+"/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/Kalaba-Project/16-K5"
serie=repertoire+"/"
#########################IMPORTS############################################
import codecs, optparse
import re, random
import sys,os,time
import string
import yaml
import ParFuMor as PFM
from ParFuMor import *
import pickle

In [1101]:
debug=0
print_glose=True
print_cloze=True
print_radicaux=True
numeros={'1':'Un','2':'Deux','3':'Trois','4':'Quatre','5':'Cinq'}
personnes={'1sg':'UnSg','2sg':'DeuxSg','3sg':'TroisSg','1du':'UnDu','2du':'DeuxDu','3du':'TroisDu','1pl':'UnPl','2pl':'DeuxPl','3pl':'TroisPl'}

commandGrapho="\\newcommand{\\%s}{\\strutgb{0pt}\\grapho{%s}}"
#commandGrapho="\\newcommand{\\%s}{\\strutgb{0pt}{\\dn %s}}"
#commandGrapho=u"\\newcommand{\\%s}{\\strutgb{0pt}{%s}}"
commandPhono=u"\\newcommand{\\%sP}{\\textipa{%s}}"
commandGlose=u"\\newcommand{\\%sG}{\\textGlose{%s}}"

In [1102]:
with open(serie+"Phonology.yaml", 'r') as stream:
    phonology=yaml.load(stream)
with open(serie+"Stems.yaml", 'r') as stream:
    stems=yaml.load(stream)
with open(serie+"MorphoSyntax.yaml", 'r') as stream:
    morphosyntax=yaml.load(stream)
with open(serie+"Hierarchie-S2.pkl", 'rb') as input:
   PFM.hierarchieCF = pickle.load(input)
with open(serie+"Lexique-S2.pkl", 'rb') as input:
   PFM.lexique = pickle.load(input)
with open(serie+"Regles-S2.pkl", 'rb') as input:
   PFM.regles = pickle.load(input)

In [1103]:
PFM.lexique.lexemes[u'dans']

sir, PREP, DANS
		sir :
			PREP:	sir	dans	sir	\textRadical{sir}

#### Définition des segments

In [1104]:
consonnes=phonology["consonnes"]
voyelles=phonology["voyelles"]
gabarits=phonology["gabarits"]
derives=phonology["derives"]
nom_classe=phonology["nom_classe"]
nom_apo=phonology["apophonies"]
nom_mut=phonology["mutations"]
syllabes=phonology["syllabes"]

#### Attention aux correspondances pour les syllabes
YAML interprète la clé no comme False

In [1105]:
def parse_grapho(graphie):
#    chunks=re.findall(r"([ptkbdgmnNfsSvzjrlyv]?[aeiou]?)|[aeiou]|[ptkbdgmnNfsSvzjrlyv]|[.…,;!?:—–()\[\]\/# ""«»<>]", graphie)
#    chunks=re.findall(r"([ptkbdgmnNfsSvzZjwrlyv]?[aeiou]?|[.…,;!?:—–()\[\]\/#""«»<>]+|.*)", graphie,re.U)
    chunks=re.findall(ur"\s+|[ptkbdgmnNfsSvzZjwrlyv]?[aeiou]?", graphie,re.U)
    result=[]
#    print chunks
    for chunk in chunks:
#        print [chunk],syllabes.keys()
        if chunk in syllabes.keys():
            result.append(syllabes[chunk])
        else:
            result.append(chunk)
    return "".join(result)

In [1106]:
def parse_cloze(glose):
    if debug: print [glose]
    chunks=re.findall(r"\\cacherGloses{([^}]*)?}|(\w+)", glose,re.UNICODE)
    result=[]
    for chunk in chunks:
        result.extend([x for x in chunk if x!=""])
    return u"%s;"%len(result)+";".join(result)

In [1107]:
#grapho=recoder("SviNaNeNNoNN",translit)
#grapho,parse_grapho(grapho)

In [1108]:
if "Cas" in morphosyntax:
    casSyntagmes=morphosyntax["Cas"]
else:
    casSyntagmes=""
lexiquePrepositions=[stems["PREP"][x][0] for x in stems["PREP"]]
casPreposition={}
for preposition in lexiquePrepositions:
    prep=preposition.upper()
    if casSyntagmes and prep in casSyntagmes:
        casPreposition[preposition]=casSyntagmes[prep]
    elif casSyntagmes and "PREP" in casSyntagmes:
        casPreposition[preposition]=casSyntagmes["PREP"]
    else:
        casPreposition[preposition]=""
glosePREP={x:u"%s%s"%(x.upper(),cacherGloses("["+casPreposition[x].strip("+").capitalize()+"]")) for x in casPreposition if "+" in casPreposition[x]}

In [1109]:
glosePREP

{u'apr\xe8s': u'APR\xc8S\\cacherGloses{[Obl]}',
 'avec': u'AVEC\\cacherGloses{[Obl]}',
 'dans': u'DANS\\cacherGloses{[Obl]}',
 'de': u'DE\\cacherGloses{[Obl]}',
 u'derri\xe8re': u'DERRI\xc8RE\\cacherGloses{[Obl]}',
 'devant': u'DEVANT\\cacherGloses{[Obl]}',
 'entre': u'ENTRE\\cacherGloses{[Obl]}',
 'sur': u'SUR\\cacherGloses{[Obl]}',
 'vers': u'VERS\\cacherGloses{[Obl]}',
 u'\xe0': u'\xc0\\cacherGloses{[Obl]}'}

In [1110]:
try:
    __IPYTHON__ 
    ipython=True
except: 
    ipython=False

version=os.path.basename("__file__")
time_stamp='%s' % time.strftime("%y%m%d-%H%M")
print "%% version : "+version
print "%% traitement : "+time_stamp

if ipython or True:
#    lexeme_nom="lexemes.txt"
#    phrase_nom="phrases.txt"
    pass
else:
    parser=optparse.OptionParser()
    parser.add_option("-o", "--out", dest="outfile", action="store_true", help="write to FILE")
    parser.add_option("-c", "--cloze", dest="print_cloze", action="store_true", help="write a CLOZE FILE")
    parser.add_option("-l", "--lexicon", dest="print_lexique", action="store_true", help="append a lexicon")
    parser.add_option("-r", "--roots", dest="print_racines", action="store_true", help="append a root list")

    (options, args) = parser.parse_args()
    lexeme_nom=args[0]
    phrase_nom=args[1]

%% version : __file__
%% traitement : 160904-2037


In [1111]:
def recoder(chaine,table,boolUTF8=True):
    if type(chaine)==str:
        temp=unicode(chaine.decode('utf8')).translate(table)
        result=temp
    elif type(chaine)==unicode:
        result=chaine.translate(table)
    if boolUTF8:
        return result
    else:
        return result.encode("utf8")
#translit=string.maketrans(u'iueoaftgzZvjkSpN',u'tgazpHTGZJVkXyxI')

In [1112]:
phonoIn =  unicode(phonology["translations"]["grapho"]["in"])
graphoIn = [ord(char) for char in phonoIn]
graphoOut = unicode(phonology["translations"]["grapho"]["out"])
translit = dict(zip(graphoIn, graphoOut))

In [1113]:
accentedIn = unicode(phonology["translations"]["deaccent"]["in"])
deaccentIn = [ord(char) for char in accentedIn]
deaccentOut = unicode(phonology["translations"]["deaccent"]["out"])
deaccent = dict(zip(deaccentIn, deaccentOut))

deligatures=phonology["translations"]["deligatures"]

In [1114]:
#translit

In [1115]:
tipaIn = unicode(phonology["translations"]["ipa"]["in"])
ipaIn = [ord(char) for char in tipaIn]
ipaOut = unicode(phonology["translations"]["ipa"]["out"])
toipa = dict(zip(ipaIn, ipaOut))

In [1116]:
tableaux={}
gloseClozes={}
declarations=[]
for categorie in PFM.lexique.catLexeme:
    if not categorie in tableaux:
        tableaux[categorie]=set()
        gloseClozes[categorie]=[]
    if verbose: print categorie
    for lexeme in PFM.lexique.catLexeme[categorie]:
        if verbose: print PFM.lexique.lexemes[lexeme]
        for case in PFM.lexique.lexemes[lexeme].paradigme.cases:
            caseGlose=case.glose
            if categorie in PFM.categoriesMajeures:
                nom=PFM.lexique.lexemes[lexeme].nom
            else:
                nom=PFM.lexique.lexemes[lexeme].nom.upper()
                gloseCase=case.glose
                if categorie=="PREP" and gloseCase in glosePREP:
                    caseGlose=glosePREP[gloseCase]
                elif categorie=="PREP":
                    caseGlose=caseGlose.upper()
#                print caseGlose
            ref=PFM.modifierGlose(nom,case.sigma,"ref")
            ref=recoder(ref,deaccent)
            for ligature in deligatures:
                ref=ref.replace(ligature,deligatures[ligature])
            for pers in personnes:
                ref=ref.replace(pers,personnes[pers])
            for num in numeros:
                ref=ref.replace(num,numeros[num])
#            phono=case.forme
#            print ("lexeme",lexeme)
            grapho=chaine2utf8(re.sub(ur"\s+",ur"",recoder(parse_grapho(case.forme),translit)))
            if print_radicaux:
                phono=case.detoure.replace(" ","")
            else:
                phono=case.forme.replace(" ","")
#            print [ref,grapho]
            declarations.append(commandGrapho%(ref,grapho))
            declarations.append(commandPhono%(ref,phono))
            declarations.append(commandGlose%(ref,caseGlose))
            if print_glose:
                tableaux[categorie].add("\\"+ref+" & \\"+ref+"P & \\"+ref+"G \\\\")
            else:
                tableaux[categorie].add("\\"+ref+" & \\"+ref+"P & \\blanc{\\"+ref+"G} \\\\")
            if print_cloze:
                vedette=nom.split(".")[0]
                gloses=parse_cloze(case.glose)
                if debug: print [grapho, recoder(phono,toipa)]
                try:
                    cloze=u";".join([ref,vedette,categorie,recoder(phono,toipa),recoder(case.decoupe,toipa),case.sigma,gloses])
                except NameError:
                    cloze=u";".join([ref,vedette,categorie,recoder(phono,toipa),gloses])
                gloseClozes[categorie].append(cloze)
    

In [1117]:
if print_radicaux:
    nomDeclaration="Declarations-Radicaux.tex"
    nomTableaux="Tableaux-Radicaux.yaml"
else:
    nomDeclaration="Declarations.tex"
    nomTableaux="Tableaux.yaml"

In [1118]:
with codecs.open(serie+nomDeclaration, 'wb', encoding='utf8') as output:
    for declaration in declarations:
#        print type(declaration),declaration
        output.write(declaration+"\n")

In [1119]:
if print_cloze:
    with codecs.open(serie+"Clozes.txt", 'wb', encoding='utf8') as output:
        for categorie in gloseClozes:
            output.write("#\t"+categorie+"\n#\n#\n")
            for cloze in gloseClozes[categorie]:
                output.write(cloze+"\n")
            output.write("#\n#\n#\n")

In [1120]:
yaml.safe_dump(tableaux, file(serie+nomTableaux, 'w'), encoding='utf-8', allow_unicode=True)

In [1121]:
#gloseClozes

In [1122]:
[l for l in declarations if "Kaleb" in l]

[u'\\newcommand{\\KalebASgErg}{\\strutgb{0pt}\\grapho{kaleLsi}}',
 u'\\newcommand{\\KalebASgErgP}{\\textipa{\\textRadical{kaleb}si}}',
 u'\\newcommand{\\KalebASgErgG}{\\textGlose{Kaleb\\cacherGloses{.N2.N21.A}\\cacherGloses{-N21.Erg}}}',
 u'\\newcommand{\\KalebASgAbs}{\\strutgb{0pt}\\grapho{kaleLke}}',
 u'\\newcommand{\\KalebASgAbsP}{\\textipa{\\textRadical{kaleb}ke}}',
 u'\\newcommand{\\KalebASgAbsG}{\\textGlose{Kaleb\\cacherGloses{.N2.N21.A}\\cacherGloses{-N21.Abs}}}',
 u'\\newcommand{\\KalebASgDat}{\\strutgb{0pt}\\grapho{kaleLso}}',
 u'\\newcommand{\\KalebASgDatP}{\\textipa{\\textRadical{kaleb}so}}',
 u'\\newcommand{\\KalebASgDatG}{\\textGlose{Kaleb\\cacherGloses{.N2.N21.A}\\cacherGloses{-N21.Dat}}}',
 u'\\newcommand{\\KalebASgObl}{\\strutgb{0pt}\\grapho{kaleLmu}}',
 u'\\newcommand{\\KalebASgOblP}{\\textipa{\\textRadical{kaleb}mu}}',
 u'\\newcommand{\\KalebASgOblG}{\\textGlose{Kaleb\\cacherGloses{.N2.N21.A}\\cacherGloses{-N21.Obl.Sg}}}',
 u'\\newcommand{\\KalebADuErg}{\\strutgb{0pt}